# Using OAK for Environmental Ontologies

* author: Chris Mungall
* created: 2022-10-24

This tutorial walks through the neuro-relevant subset of the [ENVO](https://obofoundry.org/ontology/envo) as well as SWEET.

- to help understand the structure of ENVO
- to show how to do advanced OAK queries and visualization (CLI and programmatic) on ENVO
- demonstrate rudimentary text annotation

Running this notebook locally or on mybinder requires [0.1.41](https://github.com/INCATools/ontology-access-kit/releases/tag/v0.1.41) or higher

## Create an alias

For convenience we will set a bash alias within Jupyter.

The first time you run this, a copy of envo.db.gz is downloaded from S3, which may include a delay - for subsequent invocations,
the cached copy will be used

In [1]:
%alias envo runoak -i sqlite:obo:envo

## Basic lookup

We will first lookup a term by name

In [2]:
envo info biome

ENVO:00000428 ! biome                                                           


multiple terms can be entered, using either IDs or labels (later we will see how to include *extensional* terms that expand queries to sets of terms)

In [15]:
envo info archipelago "hydrothermal vent" beach

ENVO:00000220 ! archipelago
ENVO:00000215 ! hydrothermal vent
ENVO:00000091 ! beach


### hierarchical tree displays

The [Oak Tree](https://incatools.github.io/ontology-access-kit/cli.html#runoak-tree) command will show a hierarchical tree for any set of terms.

Like any graph-oriented OAK command this is parameterized by `--predicates` (`-p`), so to show only is-a:

In [4]:
envo tree biome -p i

* [] BFO:0000001 ! entity
    * [i] BFO:0000002 ! continuant
        * [i] BFO:0000004 ! independent continuant
            * [i] BFO:0000040 ! material entity
                * [i] RO:0002577 ! system
                    * [i] ENVO:01000254 ! environmental system
                        * [i] ENVO:01001110 ! ecosystem
                            * [i] **ENVO:00000428 ! biome**


note the default output is markdown-friendly, so this can be pasted into a GitHub issue (or a Jupyter cell) and it will render, like this:

* [] BFO:0000001 ! entity
    * [i] BFO:0000002 ! continuant
        * [i] BFO:0000004 ! independent continuant
            * [i] BFO:0000040 ! material entity
                * [i] RO:0002577 ! system
                    * [i] ENVO:01000254 ! environmental system
                        * [i] ENVO:01001110 ! ecosystem
                            * [i] **ENVO:00000428 ! biome**
                            
We can also have the tree command show descendants (using `--down`).

The results can sometimes lead to "path explosions", we can control this by setting a maximum number of hops using `--max-hops`:

In [16]:
envo tree biome -p i --down --max-hops 2

* [] ENVO:01000254 ! environmental system
    * [i] ENVO:01001110 ! ecosystem
        * [i] **ENVO:00000428 ! biome**
            * [i] ENVO:01001838 ! arid biome
            * [i] ENVO:01001837 ! subalpine biome
            * [i] ENVO:01001836 ! montane biome
                * [i] ENVO:01000223 ! montane savanna biome
                * [i] ENVO:01000216 ! montane shrubland biome
            * [i] ENVO:01001835 ! alpine biome
                * [i] ENVO:01001505 ! alpine tundra biome
            * [i] ENVO:01001834 ! subpolar biome
            * [i] ENVO:01001832 ! subtropical biome
                * [i] ENVO:01001833 ! mediterranean biome
                    * [i] ENVO:01000229 ! mediterranean savanna biome
                    * [i] ENVO:01000217 ! mediterranean shrubland biome
                    * [i] ENVO:01000208 ! mediterranean woodland biome
                    * [i] ENVO:01000047 ! mediterranean sea biome
                        * [i] ENVO:01000857 ! temperate mediterranean sea 

## Search

In addition to specifying exact terms, you can specify *query terms* as inputs to most commands.

One form of query is a search query, which has structure:

`PREDICATE-CODE SEARCH-SYMBOL SEARCH-TERM`

For example, to search

- for any matching *label*, (code `l`)
- using a *substring* match (code `~`)
- for the string "oak"

We can do:

In [6]:
envo info l~oak

ENVO:01000239 ! oak woodland


In [18]:
envo info t~deciduous

ENVO:00000109 ! woodland area
ENVO:00002989 ! obsolete Temperate broad-leaf forests or woodlands and subpolar deciduous thickets
ENVO:00002990 ! obsolete Tropical dry or deciduous forests (including Monsoon forests) or woodlands
ENVO:01000385 ! temperate deciduous broadleaf forest
ENVO:01000386 ! temperate deciduous needleleaf forest
ENVO:01000387 ! tropical deciduous broadleaf forest
ENVO:01000388 ! tropical semi-deciduous broadleaf forest
ENVO:01000816 ! area of deciduous forest
PATO:0001730 ! deciduous (generic)
PATO:0001731 ! deciduous (plant)
PATO:0001732 ! non-deciduous (any body part)
PATO:0001734 ! semi-deciduous(plant)


In [20]:
envo info 'l/forest.*biome$'

ENVO:00000875 ! obsolete Temperate coniferous forest biome
ENVO:00000876 ! obsolete Temperate broadleaf and mixed forest biome
ENVO:00000878 ! obsolete Mediterranean forests, woodlands, and shrub biome
ENVO:00000879 ! obsolete Tropical and subtropical moist broadleaf forest biome
ENVO:00000880 ! obsolete Tropical and subtropical dry broadleaf forest biome
ENVO:00005806 ! obsolete Tropical and subtropical coniferous forest biome
ENVO:01000174 ! forest biome
ENVO:01000196 ! coniferous forest biome
ENVO:01000197 ! broadleaf forest biome
ENVO:01000198 ! mixed forest biome
ENVO:01000199 ! mediterranean forest biome
ENVO:01000200 ! tropical broadleaf forest biome
ENVO:01000201 ! subtropical broadleaf forest biome
ENVO:01000202 ! temperate broadleaf forest biome
ENVO:01000209 ! subtropical coniferous forest biome
ENVO:01000210 ! tropical coniferous forest biome
ENVO:01000211 ! temperate coniferous forest biome
ENVO:01000212 ! temperate mixed forest biome
ENVO:01000225 ! subtropical dry broadl

In [23]:
envo info .desc//p=i biome .and .desc//p=i,p ocean

ENVO:01000048 ! ocean biome


## Visualization

The [Oak Viz](https://incatools.github.io/ontology-access-kit/cli.html#runoak-viz) can be used to create flexible configurable visualizations centered around a set of input terms

In [1]:
envo viz "oak woodland" -o output/oak-woodland.png

SyntaxError: invalid syntax (2708209757.py, line 1)

![img](output/oak-woodland.png)

## Accessing SWEET

SWEET is also indexed in the SemSQL index, so we can download ready-made sqlite files

(the SWEET owl files can also be accessed directly using OAK, but we do not cover this in this tutorial)

First we will set up a Jupyter alias for convenience

In [2]:
%alias sweet runoak -i sqlite:obo:sweetAll

In [9]:
sweet search l~biome

sweet:realmBiolBiome/Biome ! biome
sweet:realmBiolBiome/TerrestrialBiome ! terrestrial biome


In [24]:
sweet descendants sweet:realmBiolBiome/Biome

sweet:realmBiolBiome/Anthromes ! anthromes
sweet:realmBiolBiome/Barren ! barren
sweet:realmBiolBiome/Biome ! biome
sweet:realmBiolBiome/Biota ! biota
sweet:realmBiolBiome/Croplands ! croplands
sweet:realmBiolBiome/DenseSettlements ! dense settlements
sweet:realmBiolBiome/Forested ! forested
sweet:realmBiolBiome/Rangelands ! rangelands
sweet:realmBiolBiome/SparseTrees ! sparse trees
sweet:realmBiolBiome/Terrain ! terrain
sweet:realmBiolBiome/TerrestrialBiome ! terrestrial biome
sweet:realmBiolBiome/Villages ! villages
sweet:realmBiolBiome/Wildlands ! wildlands


In [32]:
sweet tree .desc biome

* [] sweet:reprSciComponent/SystemComponent ! system component
    * [i] sweet:reprSciComponent/Environment ! environment
        * [i] **sweet:realmBiolBiome/Biome ! biome**
            * [i] **sweet:realmBiolBiome/TerrestrialBiome ! terrestrial biome**
                * [i] **sweet:realmBiolBiome/Rangelands ! rangelands**
                * [i] **sweet:realmBiolBiome/Terrain ! terrain**
                * [i] **sweet:realmBiolBiome/Wildlands ! wildlands**
                    * [i] **sweet:realmBiolBiome/SparseTrees ! sparse trees**
                    * [i] **sweet:realmBiolBiome/Barren ! barren**
                * [i] **sweet:realmBiolBiome/DenseSettlements ! dense settlements**
                * [i] **sweet:realmBiolBiome/Villages ! villages**
                * [i] **sweet:realmBiolBiome/Biota ! biota**
                * [i] **sweet:realmBiolBiome/Anthromes ! anthromes**
                * [i] **sweet:realmBiolBiome/Forested ! forested**
                * [i] **sweet:realmBiolBiome/Cr

## Accessing ENVO through Bioportal

In [19]:
%alias bpenvo runoak -i bioportal:envo

In [21]:
bpenvo annotate "Peatlands have acted as net CO2 sinks. Rock biome terrestrial biome"

match_type: PREF
object_id: ENVO:00001995
object_label: rock
object_source: https://data.bioontology.org/ontologies/ENVO
subject_end: 43
subject_label: ROCK
subject_start: 40

---
match_type: PREF
object_id: ENVO:00000428
object_label: biome
object_source: https://data.bioontology.org/ontologies/ENVO
subject_end: 49
subject_label: BIOME
subject_start: 45

---
match_type: PREF
object_id: ENVO:00000428
object_label: biome
object_source: https://data.bioontology.org/ontologies/ENVO
subject_end: 67
subject_label: BIOME
subject_start: 63

---
match_type: PREF
object_id: ENVO:00000446
object_label: terrestrial biome
object_source: https://data.bioontology.org/ontologies/ENVO
subject_end: 67
subject_label: TERRESTRIAL BIOME
subject_start: 51

---


In [27]:
%alias agroportal runoak -i agroportal:

In [33]:
agroportal annotate "Peatlands have acted as CO2 sinks"

match_type: SYN
object_id: http://www.yso.fi/onto/afo/p122
object_label: bogs
object_source: http://data.agroportal.lirmm.fr/ontologies/AFO
subject_end: 9
subject_label: PEATLANDS
subject_start: 1

---
match_type: PREF
object_id: http://id.agrisemantics.org/gacs/C10734
object_label: peatlands
object_source: http://data.agroportal.lirmm.fr/ontologies/GACS
subject_end: 9
subject_label: PEATLANDS
subject_start: 1

---
match_type: PREF
object_id: http://aims.fao.org/aos/agrovoc/c_13929
object_label: peatlands
object_source: http://data.agroportal.lirmm.fr/ontologies/AGROVOC
subject_end: 9
subject_label: PEATLANDS
subject_start: 1

---
match_type: PREF
object_id: http://www.yso.fi/onto/afo/p902
object_label: peatlands
object_source: http://data.agroportal.lirmm.fr/ontologies/AFO
subject_end: 9
subject_label: PEATLANDS
subject_start: 1

---
match_type: SYN
object_id: https://www.tno.nl/agrifood/ontology/common-greenhouse-ontology#CO2
object_label: Carbon Dioxide
object_source: http://data.ag